In [ ]:
from helpers.db.db_setup_methods import *
from helpers.db.db_query_methods import *

from helpers.db.db_helper_methods import *

record_size = 2_000_000

init_db(record_size)


This is just to try and playaround with the order by and see how indexing influeces them. At first i just want to setup something to see the order and what indexing would imfluence the query.

the assumption would be that if you have a index with the key of a column you are ordering by that you should instead of having to load everything into memory and then sort it then it should be able to just use the index to avoid the sort process.

In [ ]:
drop_non_clustered_indexes("products", get_database_name())

query = """
SELECT *
FROM products
WHERE size = 'Large'
ORDER BY price ASC;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN " + query, database=get_database_name(), print_as_df=True, show_metrics=False)

In [ ]:
drop_non_clustered_indexes("products", get_database_name())

query = """
CREATE INDEX idx_size_price ON products (size, price ASC);
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=False)

query = """
SELECT *
FROM products
WHERE size = 'Large'
ORDER BY price ASC;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN " + query, database=get_database_name(), print_as_df=True, show_metrics=False)

In [ ]:
drop_non_clustered_indexes("products", get_database_name())


query = """
CREATE INDEX idx_price ON products (price);
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=False)

query = """
SELECT *
FROM products
WHERE size = 'Large'
ORDER BY price ASC;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN " + query, database=get_database_name(), print_as_df=True, show_metrics=False)

In [ ]:
drop_non_clustered_indexes("products", get_database_name())

query = """
CREATE INDEX idx_price_size ON products (price, size);
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=False)

query = """
SELECT *
FROM products
WHERE size = 'Large'
ORDER BY price ASC;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN " + query, database=get_database_name(), print_as_df=True, show_metrics=False)

So something interresting is that at a high record count of 2M the index of size, price is acutally taking twice as long as the other quries but at a lower dataamount like 20K then it is faster. 
I want to test why that is, at first i am thinking that maybe it has something to do with it fetching like around 5-10 percent of the db with the where size = 'large'. So i want to test it on price instead

In [ ]:
drop_non_clustered_indexes("products", get_database_name())

query = """
SELECT *
FROM products
WHERE price = 999
ORDER BY size ASC;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN " + query, database=get_database_name(), print_as_df=True, show_metrics=False)

In [ ]:
drop_non_clustered_indexes("products", get_database_name())

query = """
CREATE INDEX idx_size_price ON products (price, size ASC);
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=False)

query = """
SELECT *
FROM products
WHERE price = 999
ORDER BY size ASC;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN " + query, database=get_database_name(), print_as_df=True, show_metrics=False)

This was faster but now i am thinking maybe it has to do with the I/O read of the data so lets instead turn it into being able to do a single index read.

In [ ]:
drop_non_clustered_indexes("products", get_database_name())

query = """
SELECT size, price
FROM products
WHERE size = 'Large'
ORDER BY price ASC;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN " + query, database=get_database_name(), print_as_df=True, show_metrics=False)

In [ ]:
drop_non_clustered_indexes("products", get_database_name())

query = """
CREATE INDEX idx_size_price ON products (size, price ASC);
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=False)

query = """
SELECT size, price
FROM products
WHERE size = 'Large'
ORDER BY price ASC;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN " + query, database=get_database_name(), print_as_df=True, show_metrics=False)

Yeah, now it is faster. So it had to do with eventhough it didn't have to do the sorting the sorting was quicker than having to do the indivdual i/o reads of the data to fetch every value for the select * part.

It is funny that the index acutally made it slower than just a full scan.

So also the reason why the quries where i did price = 999 order by size the reason why the index made it faster was possibly just the amount of records that it fetched.